# Programación Entera: Heurísticas

## Descripción

[TODO]

## Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
## Fecha

  * Mayo de 2018


## Contenidos
  
  * [Problema de Transporte](#Problema-de-Transporte)
  * [Problema de Asignación](#Problema-de-Asignación)
    * [Asignación Clásico](#Asignación-Clásico)
    * [Asignación Generalizado](#Asignación-Generalizado)
  * [Problema de la Mochila](#Problema-de-la-Mochila)
    * [Mochila Simple](#Mochila-Simple)
    * [Mochila Múltiple](#Mochila-Múltiple)
    * [Mochila Multidimensional](#Mochila-Multidimensional)

In [ ]:
library(lpSolve)
library(magrittr)

In [ ]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

## Funciones de Apoyo

In [ ]:
dataset.folder <- "./data"

In [ ]:
ReadDataSet <- function() {
    
}

In [ ]:
Solve <- function(direction = "max", f.obj, f.con, f.rhs, f.dir = rep("<=", nrow(f.con)), ...) {
    lp(direction, f.obj, f.con, f.dir, f.rhs, ...)
}

## Problema de Transporte

#### Descripción:
TODO

#### Modelo:
TODO

In [ ]:
TransportationExact <- function(demand, capacity, cost, direction = "min", ...) {
    m <- length(capacity)
    n <- length(demand)    
    
    conditions <- matrix(0, nrow = n + m, ncol = n * m)
    for (i in 1:m) {
        conditions[i, seq(n) + (i - 1) * n] <- 1
    }
    for (i in (m+1):(n+m)) {
        conditions[i, seq(i - m, m * n, by = n)] <- 1
    }
    res <- c(rep("<=", m), rep(">=", n))
    return(Solve(direction, cost, conditions, c(capacity, demand), res, ...))
}

#### Heurísticas:
TODO

In [ ]:
TransportationGreedy <- function(demand, capacity, cost) {
    m <- length(capacity)
    n <- length(demand) 
    sol <- list(solution = matrix(0, nrow = m, ncol = n), 
                objval = 0, slack = capacity)
    while (!all(demand == 0)) {
        ij <- which(cost == min(cost[sol$slack > 0, demand > 0]), arr.ind = TRUE)
        ij.slice <- which(sol$slack[ij[, 1]] > 0 & demand[ij[, 2]] > 0)[1]
        i <- ij[ij.slice, 1]
        j <- ij[ij.slice, 2]
        inc <- min(sol$slack[i], demand[j])    
        
        sol$solution[i, j] <- sol$solution[i, j] + inc
        sol$slack[i] <- sol$slack[i] - inc
        demand[j] <- demand[j] - inc
        sol$objval <- sol$objval + cost[i, j] * inc
    }
    return(sol)
}

### Ejemplos

In [ ]:
transportation.problem.1 <- list(
    demand = c(45, 20, 30, 30),
    capacity = c(35, 60, 48),
    cost = matrix(c(8, 6, 10, 9,
                    9, 12,13, 7,
                    14,9, 16, 5), nrow = 3, byrow = TRUE)
)

In [ ]:
a <- TransportationExact(transportation.problem.1$demand,
                    transportation.problem.1$capacity,
                    transportation.problem.1$cost)

matrix(a$solution, 
       nrow = 3, byrow =TRUE)
print(a$objval)

In [ ]:
TransportationGreedy(transportation.problem.1$demand,
                    transportation.problem.1$capacity,
                    transportation.problem.1$cost)

## Problema de Asignación

### Asignación Clásico

#### Descripción:
TODO

#### Modelo:
TODO

In [ ]:
AssignmentExact <- function(cost, ...) {
    return(TransportationExact(rep(1, dim(cost)[1]), rep(1, dim(cost)[2]), cost, ...))
}

#### Heurísticas:
TODO

In [ ]:
AssignmentGreedy <- function(cost) {    
    return(TransportationGreedy(rep(1, dim(cost)[1]), rep(1, dim(cost)[2]), cost))
}

In [ ]:
GreedyLocalSearchImprovement <- function(change.1, change.2, cost) {
    return((cost[change.1[, drop = FALSE]] + cost[change.2[, drop = FALSE]]) -
           (cost[change.1[1], change.2[2]] + cost[change.2[1], change.1[2]]))
}

In [ ]:
GreedyLocalSearchBest <- function(cost, changes) {
    best = list(v = 0, i = 0, j = 0)
    for (i in 1:nrow(changes)) {
        for(j in 1:nrow(changes)) {
            v.temp <- GreedyLocalSearchImprovement(changes[i, ,drop = FALSE], 
                                                   changes[j, , drop = FALSE],
                                                   cost)
            if (best$v < v.temp) {
                best$v <- v.temp
                best$i <- i
                best$j <- j
            }
        }
    }
    return(best)
}

In [ ]:
AssignmentGreedyLocalSearch <- function(cost) {    
    sol <- AssignmentGreedy(cost)
        
    changes <- which(sol$solution != 0, arr.ind = TRUE)
    best <- GreedyLocalSearchBest(cost, changes)
    while(best$v > 0) {

        sol$solution[changes[best$i, , drop = FALSE]] <- 0
        sol$solution[changes[best$j, , drop = FALSE]] <- 0
        
        temp <- changes[best$i, 2]
        changes[best$i, 2] <- changes[best$j, 2]
        changes[best$j, 2] <- temp
        
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 1
        sol$solution[changes[best$j, , drop = FALSE]] <- 1
        
        sol$objval <- sol$objval - best$v
        
        best <- GreedyLocalSearchBest(cost, changes)
    }
    return(sol)
}

### Ejemplos

In [ ]:
(matriz.15x15.1 <- as.matrix(read.csv("./data/assignment/matriz_15x15_1.csv", header = FALSE)))

In [ ]:
a <- AssignmentExact(matriz.15x15.1)
matrix(a$solution, nrow = dim(matriz.15x15.1)[1])
a$objval

In [ ]:
AssignmentGreedy(matriz.15x15.1)

In [ ]:
AssignmentGreedyLocalSearch(matriz.15x15.1)

In [ ]:
(matriz.15x15.2 <- as.matrix(read.csv("./data/assignment/matriz_15x15_2.csv", header = FALSE)))

In [ ]:
a <- AssignmentExact(matriz.15x15.2)
matrix(a$solution, nrow = dim(matriz.15x15.2)[1])
a$objval

In [ ]:
AssignmentGreedy(matriz.15x15.2)

In [ ]:
AssignmentGreedyLocalSearch(matriz.15x15.2)

In [ ]:
(matriz.15x15.3 <- as.matrix(read.csv("./data/assignment/matriz_15x15_3.csv", header = FALSE)))

In [ ]:
a <- AssignmentExact(matriz.15x15.3)
matrix(a$solution, nrow = dim(matriz.15x15.3)[1])
a$objval

In [ ]:
AssignmentGreedy(matriz.15x15.3)

In [ ]:
AssignmentGreedyLocalSearch(matriz.15x15.3)

### Asignación Generalizado

In [ ]:
ReadGeneralizedAssignment <- function(file.name) {
    con <- 
        file.name %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit(" ") %>%
        unlist() %>%
        as.integer()
    close(con)
    list(
        m = raw[1],
        n = raw[2],
        cost = matrix(raw[3:(2 + raw[1] * raw[2])], 
                      nrow = raw[1], ncol = raw[2], byrow = TRUE),
        resource = matrix(raw[(3 + raw[1] * raw[2]):(2 + 2 * raw[1] * raw[2])], 
                          nrow = raw[1], ncol = raw[2], byrow = TRUE),
        capacity = raw[(3 + 2 * raw[1] * raw[2]):length(raw)]
        
    ) %>%
    return()
}

#### Descripción:
TODO

#### Modelo:
TODO

In [ ]:
GeneralizedAssignmentExact <- function(problem.statement, direction = "max", ...) {    
    
    m <- problem.statement$m
    n <- problem.statement$n
    cost <- problem.statement$cost
    resource <- problem.statement$resource
    capacity <- problem.statement$capacity
    demand <- rep(1, n)
    
    
    conditions <- matrix(0, nrow = n + m, ncol = n * m)
    for (i in 1:m) {
        conditions[i, seq(n) + (i - 1) * n] <- resource[i, ]
    }
    for (i in (m + 1):(n + m)) {
        conditions[i, seq(i - m, m * n, by = n)] <- 1
    }
    res <- c(rep("<=", m), rep("=", n))
    return(Solve(direction, t(cost), conditions, c(capacity, demand), res, all.bin = TRUE, ...))
}

#### Heurísticas:
TODO

In [ ]:
GeneralizedAssignmentGreedy <- function(p) {    
    sol <- list(solution = matrix(0, nrow = p$m, ncol = p$n), 
                objval = 0, slack = p$capacity)
    while(!all(colSums(sol$solution) == 1)) {
        max.resource <- max(p$capacity - p$resource[,colSums(sol$solution) == 0])
        
        ij <- which(p$capacity - p$resource == max.resource, arr.ind = TRUE)
        ij <- ij[colSums(sol$solution[, ij[,2], drop = FALSE]) == 0, , drop = FALSE]
        ij <- ij[which.max(p$cost[ij]), ,drop = FALSE]
        
        sol$solution[ij[1], ij[2]] <- 1
        sol$slack[ij[1]] <- sol$slack[ij[1]] - p$resource[ij[1], ij[2]]
        sol$objval <- sol$objval + p$cost[ij[1], ij[2]]
    }
    return(sol)
}

In [ ]:
AssignmentGreedyLocalSearchImprovement <- function(change.1, change.2, cost) {
    return( - (cost[change.1[, drop = FALSE]] + cost[change.2[, drop = FALSE]]) +
              (cost[change.1[1], change.2[2]] + cost[change.2[1], change.1[2]]))
}

In [ ]:
AssignmentGreedyLocalSearchBest <- function(p, changes, slack) {
    best = list(v = 0, i = 0, j = 0)
    for (i in 1:nrow(changes)) {
        for(j in 1:nrow(changes)) {
            if ((slack[changes[i, , drop = FALSE][1]] + p$resource[changes[i, , drop = FALSE]] - 
                     p$resource[changes[i, 1], changes[j, 2]] > 0) & 
                (slack[changes[j, , drop = FALSE][1]] + p$resource[changes[j, , drop = FALSE]] - 
                     p$resource[changes[j, 1], changes[i, 2]] > 0)) { 
                    
                    v.temp <- AssignmentGreedyLocalSearchImprovement(changes[i, ,drop = FALSE], 
                                                                     changes[j, , drop = FALSE],
                                                                     p$cost)
                    if (best$v < v.temp) {
                        best$v <- v.temp
                        best$i <- i
                        best$j <- j
                    }
                }
        }
    }
    return(best)
}

In [ ]:
GeneralizedAssignmentGreedyLocalSearch <- function(p) {
    sol <- GeneralizedAssignmentGreedy(p)
    
    changes <- which(sol$solution != 0, arr.ind = TRUE)
    slack <- rowSums(sol$solution * p$resource)
    best <- AssignmentGreedyLocalSearchBest(p, changes, sol$slack)
    while(best$v > 0) {
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 0
        sol$solution[changes[best$j, , drop = FALSE]] <- 0
        sol$slack[changes[best$i][1]] <- sol$slack[changes[best$i][1]] + p$resource[changes[best$i, , drop = FALSE]]
        sol$slack[changes[best$j][1]] <- sol$slack[changes[best$j][1]] + p$resource[changes[best$j, , drop = FALSE]]
        
        temp <- changes[best$i, 2]
        changes[best$i, 2] <- changes[best$j, 2]
        changes[best$j, 2] <- temp
        
        sol$solution[changes[best$i, , drop = FALSE]] <- 1
        sol$solution[changes[best$j, , drop = FALSE]] <- 1
        
        sol$slack[changes[best$i][1]] <- sol$slack[changes[best$i][1]] - p$resource[changes[best$i, , drop = FALSE]]
        sol$slack[changes[best$j][1]] <- sol$slack[changes[best$j][1]] - p$resource[changes[best$j, , drop = FALSE]]
        sol$objval <- sol$objval + best$v
        best <- AssignmentGreedyLocalSearchBest(p, changes, sol$slack)
    }
    return(sol)
}

### Ejemplos

In [ ]:
(gap2.1 <- ReadGeneralizedAssignment("./data/assignment/gap2_1.txt"))

In [ ]:
a <- GeneralizedAssignmentExact(gap2.1)
matrix(a$solution, nrow = gap2.1$m, byrow = TRUE)
a$objval

In [ ]:
GeneralizedAssignmentGreedy(gap2.1)

In [ ]:
GeneralizedAssignmentGreedyLocalSearch(gap2.1)

In [ ]:
(gap4.1 <- ReadGeneralizedAssignment("./data/assignment/gap4_1.txt"))

In [ ]:
a <- GeneralizedAssignmentExact(gap4.1)
matrix(a$solution, nrow = gap4.1$m, byrow = TRUE)
a$objval

In [ ]:
GeneralizedAssignmentGreedy(gap4.1)

In [ ]:
GeneralizedAssignmentGreedyLocalSearch(gap4.1)

In [ ]:
(gap6.1 <- ReadGeneralizedAssignment("./data/assignment/gap6_1.txt"))

In [ ]:
a <- GeneralizedAssignmentExact(gap6.1)
matrix(a$solution, nrow = gap6.1$m, byrow = TRUE)
a$objval

In [ ]:
GeneralizedAssignmentGreedy(gap6.1)

In [ ]:
GeneralizedAssignmentGreedyLocalSearch(gap6.1)

In [ ]:
(gap8.1 <- ReadGeneralizedAssignment("./data/assignment/gap8_1.txt"))

In [ ]:
# a <- GeneralizedAssignmentExact(gap8.1)
# matrix(a$solution, nrow = gap8.1$m, byrow = TRUE)
# a$objval

In [ ]:
GeneralizedAssignmentGreedy(gap8.1)

In [ ]:
GeneralizedAssignmentGreedyLocalSearch(gap8.1)

In [ ]:
(gap12.1 <- ReadGeneralizedAssignment("./data/assignment/gap12_1.txt"))

In [ ]:
# a <- GeneralizedAssignmentExact(gap12.1)
# matrix(a$solution, nrow = gap12.1$m, byrow = TRUE)
# a$objval

In [ ]:
GeneralizedAssignmentGreedy(gap12.1)

In [ ]:
GeneralizedAssignmentGreedyLocalSearch(gap12.1)

## Problema de la Mochila

#### Descripción:
TODO

### Mochila Simple

#### Descripción:
TODO

#### Modelo:
TODO

In [ ]:
KnapsackSimpleExact <- function(object, capacity) {
    
}

#### Heurísticas:
TODO

In [ ]:
KnapsackSimpleGreedy <- function(object, capacity) {
    
}

### Mochila Múltiple

#### Descripción:
TODO

#### Modelo:
TODO

In [ ]:
KnapsackMultipleExact <- function(object, capacity) {
    
}

#### Heurísticas:
TODO

In [ ]:
KnapsackMultipleGreedy <- function(object, capacity) {
    
}

### Mochila Multidimensional

#### Descripción:
TODO

#### Modelo:
TODO

In [ ]:
KnapsackMultidimensionalExact <- function(object, capacity, ...) {
    n <- length(object$value)
    m <- length(capacity)    
    return(Solve("max", object$value, t(object$weight), capacity, all.bin = TRUE, ...))
}

#### Heurísticas:
TODO

In [ ]:
KnapsackMultidimensionalGreedy <- function(object, capacity) {
    
}

### Ejemplos

In [ ]:
(knapsack.multidimensional.problem.1 <- list(
    object = list(
        value = c(12, 13, 15, 10),
        weight = matrix(c(0.5, 0.3, 0.2,
                          1.0, 0.8, 0.2,
                          1.5, 1.5, 0.1,
                          3.1, 2.5, 0.4), nrow = 4, byrow = TRUE) 
    ),
    capacity = c(3.1, 2.5, 0.4)
))

In [ ]:
a <- KnapsackMultidimensionalExact(
    knapsack.multidimensional.problem.1$object, 
    knapsack.multidimensional.problem.1$capacity
)
matrix(a$solution, nrow = 1, byrow = TRUE)
a$objval